In [359]:
import pandas as pd
%matplotlib inline
from sklearn.cluster import KMeans
import sklearn.cluster as cluster
import os
from pathlib import Path


In [360]:
data = pd.read_csv('testing_full_2nd_day_clusters_18.csv',low_memory=False)
data = data.drop(['Unnamed: 0'],axis=1)
data = data.fillna(method='bfill')

In [361]:
data.head()

,Time,IP.Source,IP.Destination,MAC.Source,MAC.Destination,Protocol,Source Port,Destination Port,TTL,Length,Win.Size,Device,Class
0,0.000000,3232235896,778643023,1865,1829,1,40234.0,5228.0,64.0,54.0,65360.0,Triby Speaker,19
1,0.000123,3232235896,778643023,1865,1829,1,40234.0,5228.0,64.0,54.0,65360.0,Triby Speaker,19
2,0.065516,878178719,3232235882,1829,1865,1,443.0,46330.0,224.0,54.0,963.0,TPLink Router Bridge LAN (Gateway),29
3,0.125605,3232235972,872491130,1865,1829,1,3063.0,443.0,100.0,54.0,3072.0,SmartThings,1
4,0.298227,778643023,3232235896,1829,1865,1,5228.0,40234.0,48.0,9.0,63063.0,TPLink Router Bridge LAN (Gateway),29


In [362]:
data['Source Port'] = pd.to_numeric(data['Source Port'], errors='coerce').fillna(0)
data['Destination Port'] = pd.to_numeric(data['Destination Port'], errors='coerce').fillna(0)

In [363]:
features_list = ['TTL','Length','Win.Size','MAC.Source','MAC.Destination']

In [364]:
def run_kmeans(cluters_numbers, features_list):
    kmeans = cluster.KMeans(n_clusters=cluters_numbers ,init="k-means++",max_iter=1000)
    kmeans = kmeans.fit(data[features_list])
    data['Clusters'] = kmeans.labels_
    return kmeans

In [365]:
features_list = ['TTL','Length','Win.Size','MAC.Source','MAC.Destination']
kmeans = run_kmeans(18,features_list)

In [366]:
def create_ground_thruth_file(ground_thruth_file_name, data_frame):
    ground_thruth_files_path = './ground_thruth_files_batch_folder_2/'
    file_name = open(ground_thruth_files_path + ground_thruth_file_name, "w")
    lines = []
    for i in range(1,len(data_frame)):
        line = "{}{}".format('obs_', i) + ",{}{}\n".format('obs_',data_frame['Device'][i])
        lines.append(line)
    file_name.writelines(lines)
    file_name.close()

In [367]:
def create_clustering_result_file(clustering_file_name, data_frame):
    clustering_file_path = './clustering_results_batch_folder_2/'
    file_name = open(clustering_file_path + clustering_file_name, "w")
    lines = []
    groups = kmeans.predict(data_frame[features_list])
    for i in range(1,len(data_frame)):
        line = "{}{}".format('obs_', i) + ",{}{}\n".format('cluster_',groups[i])
        lines.append(line)
    file_name.writelines(lines)
    file_name.close()

In [368]:
path = './testing_batches/batch_folder_2/'
files = os.listdir(path)

batch_folder = []

for f in files:
    batch_folder.append(f)

In [369]:
features_list = ['TTL','Length','Win.Size','MAC.Source','MAC.Destination']
kmeans = run_kmeans(18,features_list)

for i in range(1,len(batch_folder)):
    file_name =  path + batch_folder[i]
    df = pd.read_csv(file_name)
    clustering = "{}{}{}".format('clustering_result', i,'.csv')
    create_clustering_result_file(clustering, df)
    ground_thruth = "{}{}{}".format('ground_thruth', i,'.csv')
    create_ground_thruth_file( ground_thruth,df)